In [1]:
import requests
from bs4 import BeautifulSoup
import os
import sys
import time
import json

In [4]:
class VGMCrawler():
    BASE_URL = 'https://www.vgmusic.com/music/console/'
    archive_dir = 'archive'
    
    def __init__(self, sleep_time=0.1, log=True):
        self.sleep_time = sleep_time
        self.log = log
        self.count = 0
        
    def _request_url(self, url, doctype='html'):
        # set header
        try:
            response = requests.get(url)
        except:
            return None

        # sleep
        time.sleep(self.sleep_time)

        # return
        if doctype =='html':
            soup = BeautifulSoup(response.text, 'html.parser')
            return soup
        elif  doctype =='content':
            return response.content
        else:
            return response

    def _log_print(self, log, quite=False):
        if not quite:
            print(log)

        if self.log:
            with open("log.txt", "a") as f:
                print(log, file=f)
    
    def fetch_dirs(self, url):
        soup = self._request_url(url)
        tr_list = soup.find('table').find_all('tr')
        dir_list = []

        for i in range(3, len(tr_list)-1):
            dir_list.append(tr_list[i].find_all('td')[1].text)
        return dir_list
    
    def fetch_songs(self, url):
        soup = self._request_url(url)
        tr_list = soup.find('table').find_all('tr')
        song_list = []
        
        if len(tr_list) == 4:
            return None
        for i in range(2, len(tr_list)):
            now_tr = tr_list[i]
            if now_tr.get("class") == ['header']:
                album_name = now_tr.text.strip('\n')
            else:
                if not now_tr.td.get('colspan'):

                    # get info
                    info = now_tr.td.text.split('\n')
                    song_midi = now_tr.a['href']
                    song_name = info[0]
                    song_size = info[1]
                    song_author = info[3]
                    song_list.append({
                            'filename':song_midi, 
                            'song_name':song_name,
                            'song_size':song_size,
                            'song_author':song_author,
                            'album_name':album_name})
                    print('    |%30s |%30s |%13s |%10s |%s '%(album_name, song_name, song_size, song_author, song_midi ))
            
                    
        return song_list
    
    def crawl_songs(self, url, song_list, dir_path):
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)
            
        for idx, s in enumerate(song_list):
            sys.stdout.write('%d/%d - total: %d\n' % (idx, len(song_list), self.count))
            sys.stdout.flush()
            midi_url = url + s['filename']
    
            content = self._request_url(midi_url, doctype='content')
        
            if content is None:
                break
            
            fn = os.path.join(dir_path, s['filename'])
            with open(fn, "wb") as f:
                f.write(content)
                
            self.count += 1
                
    def crawl_archive(self):
        dir_list = self.fetch_dirs(self.BASE_URL)
        
        if not os.path.exists(self.archive_dir,):
            os.makedirs(self.archive_dir,)
            
        info = dict()
        for d in dir_list:
            root_dir = d.strip('/')
            print('{{%s}}' % root_dir)
            subdir_url = self.BASE_URL + d
            subdir_list = self.fetch_dirs(subdir_url)
            
            tmp_dict = dict()
            for sd in subdir_list:
                root_subdir = sd.strip('/')
                print('[%s]'%root_subdir)
                page_url = subdir_url + sd
                sl = self.fetch_songs(page_url)
                if sl:
                    self.crawl_songs(page_url, sl, os.path.join(self.archive_dir, root_dir, root_subdir))
                    tmp_dict[sd] = sl

            info[d] = tmp_dict
            
        with open(os.path.join(self.archive_dir,'archive.json'), "w") as f:
            json.dump(info, f)     
    
    def run(self): 
        s = time.time()
        self.crawl_archive()
        e = time.time()
        self._log_print(time.strftime("\nElapsed time: %H:%M:%S", time.gmtime(s-e)))
        self._log_print('Total %d Songs'&self.count)
        


In [ ]:
if __name__ == '__main__':
    vc = VGMCrawler()
    vc.run()

{{3do}}
[3do]
    |         Montana Jones (Japan) |                 Kitty's Theme |  19400 bytes |     G-Han |kittytheme.mid 
    |         Montana Jones (Japan) |      Overworld Theme - "Rest" |   3817 bytes |Guan van Zoggel |Rest_MJ.mid 
    |         Montana Jones (Japan) |Return The Stolen (Jazz Rock Remix) |  27288 bytes |    DoKoCo |return_the_stolen.mid 
    |                Star Control 2 |              Ur-Quan Kzer-Za  |   5764 bytes |Kirill Podzorov |urquan.mid 
    |  Super Street Fighter 2 Turbo |                           Ken |  52694 bytes |   lion589 |ken4.mid 
    | Super Street Fighter II Turbo |                 Akuma (Remix) |  27414 bytes |Stephen_Bereznicki |akumathemeremix.mid 
    | Super Street Fighter II Turbo |                         Guile |  53133 bytes |resolutezach1 |37245_Super-Street-Fighter-2--Guile.mid 
    | Super Street Fighter II Turbo |                         Intro |   9890 bytes |     Spark |ssf2t-intro.mid 
0/8 - total: 0
1/8 - total: 1
2/8 - tot

1/313 - total: 9
2/313 - total: 10
3/313 - total: 11
4/313 - total: 12
5/313 - total: 13
6/313 - total: 14
7/313 - total: 15
8/313 - total: 16
9/313 - total: 17
10/313 - total: 18
11/313 - total: 19
12/313 - total: 20
13/313 - total: 21
14/313 - total: 22
15/313 - total: 23
16/313 - total: 24
17/313 - total: 25
18/313 - total: 26
19/313 - total: 27
20/313 - total: 28
21/313 - total: 29
22/313 - total: 30
23/313 - total: 31
24/313 - total: 32
25/313 - total: 33
26/313 - total: 34
27/313 - total: 35
28/313 - total: 36
29/313 - total: 37
30/313 - total: 38
31/313 - total: 39
32/313 - total: 40
33/313 - total: 41
34/313 - total: 42
35/313 - total: 43
36/313 - total: 44
37/313 - total: 45
38/313 - total: 46
39/313 - total: 47
40/313 - total: 48
41/313 - total: 49
42/313 - total: 50
43/313 - total: 51
44/313 - total: 52
45/313 - total: 53
46/313 - total: 54
47/313 - total: 55
48/313 - total: 56
49/313 - total: 57
50/313 - total: 58
51/313 - total: 59
52/313 - total: 60
53/313 - total: 61
54/

1/9 - total: 330
2/9 - total: 331
3/9 - total: 332
4/9 - total: 333
5/9 - total: 334
6/9 - total: 335
7/9 - total: 336
8/9 - total: 337
{{mattel}}
[intellivision]
    |                      Atlantis |                        Danger |    388 bytes |Joo *Johnnyz* Buaes |Atlantis-Danger.mid 
    |                      Atlantis |                     Game Over |    640 bytes |Joo *Johnnyz* Buaes |Atlantis-Over.mid 
    |                      Atlantis |                     Start Day |    607 bytes |Joo *Johnnyz* Buaes |Atlantis-Start.mid 
    |                    Bomb Squad |                       In-Game |   3214 bytes |Michael Hayes |Bomb_Squad.mid 
    |                 Bump 'n' Jump |                    Game Start |    508 bytes |Joo *Johnnyz* Buaes |BNJ-Start.mid 
    |                 Bump 'n' Jump |                       In Game |   7370 bytes |Joo *Johnnyz* Buaes |BNJ-Game.mid 
    |                 Bump 'n' Jump |                   Next Season |    493 bytes |Joo *Johnnyz* Buaes |BNJ

1/90 - total: 339
2/90 - total: 340
3/90 - total: 341
4/90 - total: 342
5/90 - total: 343
6/90 - total: 344
7/90 - total: 345
8/90 - total: 346
9/90 - total: 347
10/90 - total: 348
11/90 - total: 349
12/90 - total: 350
13/90 - total: 351
14/90 - total: 352
15/90 - total: 353
16/90 - total: 354
17/90 - total: 355
18/90 - total: 356
19/90 - total: 357
20/90 - total: 358
21/90 - total: 359
22/90 - total: 360
23/90 - total: 361
24/90 - total: 362
25/90 - total: 363
26/90 - total: 364
27/90 - total: 365
28/90 - total: 366
29/90 - total: 367
30/90 - total: 368
31/90 - total: 369
32/90 - total: 370
33/90 - total: 371
34/90 - total: 372
35/90 - total: 373
36/90 - total: 374
37/90 - total: 375
38/90 - total: 376
39/90 - total: 377
40/90 - total: 378
41/90 - total: 379
42/90 - total: 380
43/90 - total: 381
44/90 - total: 382
45/90 - total: 383
46/90 - total: 384
47/90 - total: 385
48/90 - total: 386
49/90 - total: 387
50/90 - total: 388
51/90 - total: 389
52/90 - total: 390
53/90 - total: 391
54

1/58 - total: 429
2/58 - total: 430
3/58 - total: 431
4/58 - total: 432
5/58 - total: 433
6/58 - total: 434
7/58 - total: 435
8/58 - total: 436
9/58 - total: 437
10/58 - total: 438
11/58 - total: 439
12/58 - total: 440
13/58 - total: 441
14/58 - total: 442
15/58 - total: 443
16/58 - total: 444
17/58 - total: 445
18/58 - total: 446
19/58 - total: 447
20/58 - total: 448
21/58 - total: 449
22/58 - total: 450
23/58 - total: 451
24/58 - total: 452
25/58 - total: 453
26/58 - total: 454
27/58 - total: 455
28/58 - total: 456
29/58 - total: 457
30/58 - total: 458
31/58 - total: 459
32/58 - total: 460
33/58 - total: 461
34/58 - total: 462
35/58 - total: 463
36/58 - total: 464
37/58 - total: 465
38/58 - total: 466
39/58 - total: 467
40/58 - total: 468
41/58 - total: 469
42/58 - total: 470
43/58 - total: 471
44/58 - total: 472
45/58 - total: 473
46/58 - total: 474
47/58 - total: 475
48/58 - total: 476
49/58 - total: 477
50/58 - total: 478
51/58 - total: 479
52/58 - total: 480
53/58 - total: 481
54

...

1/849 - total: 3489
2/849 - total: 3490
3/849 - total: 3491
4/849 - total: 3492
5/849 - total: 3493
6/849 - total: 3494
7/849 - total: 3495
8/849 - total: 3496
9/849 - total: 3497
10/849 - total: 3498
11/849 - total: 3499
12/849 - total: 3500
13/849 - total: 3501
14/849 - total: 3502
15/849 - total: 3503
16/849 - total: 3504
17/849 - total: 3505
18/849 - total: 3506
19/849 - total: 3507
20/849 - total: 3508
21/849 - total: 3509
22/849 - total: 3510
23/849 - total: 3511
24/849 - total: 3512
25/849 - total: 3513
26/849 - total: 3514
27/849 - total: 3515
28/849 - total: 3516
29/849 - total: 3517
30/849 - total: 3518
31/849 - total: 3519
32/849 - total: 3520
33/849 - total: 3521
34/849 - total: 3522
35/849 - total: 3523
36/849 - total: 3524
37/849 - total: 3525
38/849 - total: 3526
39/849 - total: 3527
40/849 - total: 3528
41/849 - total: 3529
42/849 - total: 3530
43/849 - total: 3531
44/849 - total: 3532
45/849 - total: 3533
46/849 - total: 3534
47/849 - total: 3535
48/849 - total: 3536
4

379/849 - total: 3867
380/849 - total: 3868
381/849 - total: 3869
382/849 - total: 3870
383/849 - total: 3871
384/849 - total: 3872
385/849 - total: 3873
386/849 - total: 3874
387/849 - total: 3875
388/849 - total: 3876
389/849 - total: 3877
390/849 - total: 3878
391/849 - total: 3879
392/849 - total: 3880
393/849 - total: 3881
394/849 - total: 3882
395/849 - total: 3883
396/849 - total: 3884
397/849 - total: 3885
398/849 - total: 3886
399/849 - total: 3887
400/849 - total: 3888
401/849 - total: 3889
402/849 - total: 3890
403/849 - total: 3891
404/849 - total: 3892
405/849 - total: 3893
406/849 - total: 3894
407/849 - total: 3895
408/849 - total: 3896
409/849 - total: 3897
410/849 - total: 3898
411/849 - total: 3899
412/849 - total: 3900
413/849 - total: 3901
414/849 - total: 3902
415/849 - total: 3903
416/849 - total: 3904
417/849 - total: 3905
418/849 - total: 3906
419/849 - total: 3907
420/849 - total: 3908
421/849 - total: 3909
422/849 - total: 3910
423/849 - total: 3911
424/849 - 

752/849 - total: 4240
753/849 - total: 4241
754/849 - total: 4242
755/849 - total: 4243
756/849 - total: 4244
757/849 - total: 4245
758/849 - total: 4246
759/849 - total: 4247
760/849 - total: 4248
761/849 - total: 4249
762/849 - total: 4250
763/849 - total: 4251
764/849 - total: 4252
765/849 - total: 4253
766/849 - total: 4254
767/849 - total: 4255
768/849 - total: 4256
769/849 - total: 4257
770/849 - total: 4258
771/849 - total: 4259
772/849 - total: 4260
773/849 - total: 4261
774/849 - total: 4262
775/849 - total: 4263
776/849 - total: 4264
777/849 - total: 4265
778/849 - total: 4266
779/849 - total: 4267
780/849 - total: 4268
781/849 - total: 4269
782/849 - total: 4270
783/849 - total: 4271
784/849 - total: 4272
785/849 - total: 4273
786/849 - total: 4274
787/849 - total: 4275
788/849 - total: 4276
789/849 - total: 4277
790/849 - total: 4278
791/849 - total: 4279
792/849 - total: 4280
793/849 - total: 4281
794/849 - total: 4282
795/849 - total: 4283
796/849 - total: 4284
797/849 - 

    |               Mega Man Zero 3 |Title Screen - "Title III" (v1.1) |   5211 bytes |Jordan Davenport |RMZ3_Title_V1_1.mid 
    |               Mega Man Zero 3 |Twilight Desert - "For Endless Fight III" (2) |  23877 bytes |Jay Reichard |MMZ3_-_For_Endless_Fight_III.mid 
    |               Mega Man Zero 3 |Twilight Desert - "For Endless Fight III" (v1.2) |  40099 bytes |Jordan Davenport |RMZ3_Desert_V1_2.mid 
    |               Mega Man Zero 3 |Weapons Recycling Plant - "Reborn Mechanics" |  35441 bytes |Jay Reichard |MMZ3_-_Reborn_Mechanics.mid 
    |               Mega Man Zero 4 |       Area Zero - "Esperanto" |  35329 bytes |Jay Reichard |MMZ4_-_Esperanto_V1_1.mid 
    |               Mega Man Zero 4 | Boss Battle - "Nothing Beats" |  50500 bytes |Jay Reichard |MMZ4_-_Nothing_Beats.mid 
    |               Mega Man Zero 4 |Boss Battle - "Nothing Beats" (2) |  54261 bytes |King Meteor |MMZ4_Boss-KM.mid 
    |               Mega Man Zero 4 |                  Craft (v1.1) |  31575 

1/1737 - total: 4338
2/1737 - total: 4339
3/1737 - total: 4340
4/1737 - total: 4341
5/1737 - total: 4342
6/1737 - total: 4343
7/1737 - total: 4344
8/1737 - total: 4345
9/1737 - total: 4346
10/1737 - total: 4347
11/1737 - total: 4348
12/1737 - total: 4349
13/1737 - total: 4350
14/1737 - total: 4351
15/1737 - total: 4352
16/1737 - total: 4353
17/1737 - total: 4354
18/1737 - total: 4355
19/1737 - total: 4356
20/1737 - total: 4357
21/1737 - total: 4358
22/1737 - total: 4359
23/1737 - total: 4360
24/1737 - total: 4361
25/1737 - total: 4362
26/1737 - total: 4363
27/1737 - total: 4364
28/1737 - total: 4365
29/1737 - total: 4366
30/1737 - total: 4367
31/1737 - total: 4368
32/1737 - total: 4369
33/1737 - total: 4370
34/1737 - total: 4371
35/1737 - total: 4372
36/1737 - total: 4373
37/1737 - total: 4374
38/1737 - total: 4375
39/1737 - total: 4376
40/1737 - total: 4377
41/1737 - total: 4378
42/1737 - total: 4379
43/1737 - total: 4380
44/1737 - total: 4381
45/1737 - total: 4382
46/1737 - total: 43

In [ ]:
print("hi")

In [191]:
vc = VGMCrawler()
vc.fetch_songs('https://www.vgmusic.com/music/console/magnavox/odyssey/')